In [1]:
import shap
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np 
import warnings
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve
warnings.filterwarnings("ignore")
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import re
import random
import itertools
import lime
from lime import lime_tabular
from sklearn.linear_model import Ridge

In [2]:
train_num=random.sample(range(0,1000), 800)
test_num=list(set(range(0,1000))-set(train_num))
data_simulation = pd.read_csv('/ui/abv/liuzx18/project_shapley/simulation_data/simulation_0_6.csv').iloc[train_num,]
data_simulation_test = pd.read_csv('/ui/abv/liuzx18/project_shapley/simulation_data/simulation_0_6.csv').iloc[test_num,]
x_df=data_simulation[['x1','x2','x3','x4','x5','x6','x7','x8','x9','x10']]
X_=np.array(x_df)[:]
x_df_test=data_simulation_test[['x1','x2','x3','x4','x5','x6','x7','x8','x9','x10']]
X_test=np.array(x_df_test)

y_train=np.array(data_simulation[['y.binary']])
y_train=y_train.reshape(800,)


y_test=np.array(data_simulation_test[['y.binary']])
y_test=y_test.reshape(200,)
##training information:
trt_=data_simulation[['treatment']]
g_real=data_simulation[['sigpos']]
##testing information:
g_real_test=data_simulation_test[['sigpos']]




logreg = LogisticRegression()
logreg.fit(X_,trt_)
pi_x = logreg.predict_proba(X_)
pi_train=pi_x[:,1]



X_train_trt=np.where(trt_==1,1,-1)
X_train_trt=X_train_trt.reshape(800,)

dtrain = xgb.DMatrix(X_, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)


x_train_feature_pd=x_df

In [3]:

from sklearn.model_selection import KFold

EPS = 1e-12
A   = X_train_trt          # ±1 array  (n, )
X   = X_                   # feature matrix  (n, p)
Y   = y_train              # binary {0,1} outcome  (n, )
K   = 5                    # number of folds

m1_train = np.zeros_like(Y, dtype=float)
m0_train = np.zeros_like(Y, dtype=float)

kf = KFold(n_splits=K, shuffle=True, random_state=123)

for train_idx, test_idx in kf.split(X):

    # ---------- treated arm (+1) ----------
    mask_t = (A[train_idx] == 1)
    clf1 = xgb.XGBClassifier(
        max_depth=3,
        n_estimators=200,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        objective='binary:logistic',   # default, but explicit is clearer
        eval_metric='logloss'
    )
    clf1.fit(X[train_idx][mask_t], Y[train_idx][mask_t])

    # predict *probability* of class 1
    m1_train[test_idx] = clf1.predict_proba(X[test_idx])[:, 1]

    # ---------- control arm (−1) ----------
    mask_c = (A[train_idx] == -1)
    clf0 = xgb.XGBClassifier(
        max_depth=3,
        n_estimators=200,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        objective='binary:logistic',
        eval_metric='logloss'
    )
    clf0.fit(X[train_idx][mask_c], Y[train_idx][mask_c])

    m0_train[test_idx] = clf0.predict_proba(X[test_idx])[:, 1]


In [4]:
def dr_pseudo_a_bin(y, A, pi, m1, m0):
    A01   = (A + 1.0) / 2.0          # 0/1 coding
    m_A   = np.where(A == 1, m1, m0)
    denom = pi * (1.0 - pi) + EPS
    return (A01 - pi) * (y - m_A) / denom + (m1 - m0)

def dr_pseudo_w_bin(y, A, pi, m1, m0):
    A01 = (A + 1.0) / 2.0
    m_A = np.where(A == 1, m1, m0)
    c   = (1.0 - A) / 2.0 + pi * A          # same weight as your code
    phi = (A01 - pi) * (y - m_A) / (c + EPS) + (m1 - m0)
    return phi, c + EPS                     # return both φ_DR and the weight


def obj_dr_a_bin(predt, dtrain):
    y  = dtrain.get_label()
    φ  = dr_pseudo_a_bin(y, X_train_trt, pi_train, m1_train, m0_train)
    grad = -2.0 * (φ - predt)
    hess =  2.0 * np.ones_like(predt)
    return grad, hess

def eval_dr_a_bin(predt, dtrain):
    y  = dtrain.get_label()
    φ  = dr_pseudo_a_bin(y, X_train_trt, pi_train, m1_train, m0_train)
    return 'DR_A_bin_loss', float(np.mean((φ - predt) ** 2))


def obj_dr_w_bin(predt, dtrain):
    y        = dtrain.get_label()
    φ, c     = dr_pseudo_w_bin(y, X_train_trt, pi_train, m1_train, m0_train)
    grad = -2.0 / c * (φ - predt)
    hess =  2.0 / c
    return grad, hess

def eval_dr_w_bin(predt, dtrain):
    y        = dtrain.get_label()
    φ, c     = dr_pseudo_w_bin(y, X_train_trt, pi_train, m1_train, m0_train)
    return 'DR_W_bin_loss', float(np.mean(((φ - predt) ** 2) / c))

In [5]:
best_params={'learning_rate':0.005,
     'booster':'gbtree',
     'max_depth':1,
     'alpha':5,
     'min_child_weight':5,
     'tree_method':'exact',
    'subsample':0.6,
    'max_delta_step':1,
    'gamma':1,
    'max_bin':128
    }



best_epoch=80



model = xgb.train(
best_params,
dtrain,
num_boost_round=best_epoch,  # Use the best num_boost_round
obj=obj_dr_a_bin,
feval=eval_dr_a_bin)

# Predictions and Evaluation on Train Set
pred_train = model.predict(dtrain)
pred_train_prob = 1.0 / (1.0 + np.exp(-pred_train))  # Sigmoid transformation if needed
auc_train = roc_auc_score(g_real.astype(int), pred_train_prob)
print(f"Final Train AUC: {auc_train}")

# Predictions and Evaluation on Test Set
pred_test = model.predict(dtest)
pred_test_prob = 1.0 / (1.0 + np.exp(-pred_test))  # Sigmoid transformation if needed
auc_test = roc_auc_score(g_real_test.astype(int), pred_test_prob)
print(f"Final Test AUC: {auc_test}")


Final Train AUC: 0.9264253348291228
Final Test AUC: 0.9311431143114312
